In [3]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os

WHITE, BLACK = 127, 225

In [4]:
# read in image in greyscale
def getImg(image_path):
    return cv2.imread(image_path)

def getImg_bnw(image_path):
    return cv2.imread(image_path, 0)

def binarizeImg(im):
    ### a threshold in front
#     im = cv2.threshold(im, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    ### end of threshold
    
    ### code to filter out not black part first
    hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
    
    lower_black = np.array([0,0,0])
    upper_black = np.array([130,130,130])
    
    mask = cv2.inRange(hsv, lower_black, upper_black)
    res = cv2.bitwise_and(im,im, mask= mask)
    ### end of filtering
    
    im = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)

    gray = cv2.bitwise_not(im)

    ### guassian blur and otsu thresholding
    gray = cv2.GaussianBlur(gray,(5,5),0)
    thresh_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    return thresh_img

def binarizeImg1(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    gray = cv2.bitwise_not(im)

    ### guassian blur and otsu thresholding
    gray = cv2.GaussianBlur(gray,(5,5),0)
    thresh_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    cv2.imwrite('binResults.png', thresh_img)
    return thresh_img


def cvtImageToPoints(binImg):
    Xs = []
    Ys = []
    data = []
    h, w = binImg.shape
    for i in range(h):
        for j in range(w):
            if binImg[i][j] == 0:
                Xs.append(j)
                Ys.append(i)
                data.append([j,i])
    return np.asarray(data), np.asarray(Xs), np.asarray(Ys)

def getFirstComponent(image_path):
    img = getImg(image_path)
    try:
        thresh_img = binarizeImg(img)
        data, Xs, Ys = cvtImageToPoints(thresh_img)
        mean_vec = np.mean(data, axis=0)

        # Calculate the deviations from the mean
        cov_mat = (data - mean_vec).T.dot((data - mean_vec)) / (data.shape[0]-1)
        eig_vals, eig_vecs = np.linalg.eig(cov_mat)
        eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
        eig_pairs.sort(key=lambda x: x[0], reverse=True)


        vec_x = [(eig_pairs[0][1][0]+eig_pairs[0][1][0])/2, (eig_pairs[0][1][1]+eig_pairs[1][1][0])/2]
        vec_y = [(eig_pairs[1][1][0]+eig_pairs[0][1][1])/2, (eig_pairs[1][1][1]+eig_pairs[1][1][1])/2]

        first_cpts = [vec_x[0],vec_y[0]] # more robust

        return first_cpts
    
    except Exception:
        thresh_img = binarizeImg1(img)
        data, Xs, Ys = cvtImageToPoints(thresh_img)
        mean_vec = np.mean(data, axis=0)

        # Calculate the deviations from the mean
        cov_mat = (data - mean_vec).T.dot((data - mean_vec)) / (data.shape[0]-1)
        eig_vals, eig_vecs = np.linalg.eig(cov_mat)
        eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
        eig_pairs.sort(key=lambda x: x[0], reverse=True)


        vec_x = [(eig_pairs[0][1][0]+eig_pairs[0][1][0])/2, (eig_pairs[0][1][1]+eig_pairs[1][1][0])/2]
        vec_y = [(eig_pairs[1][1][0]+eig_pairs[0][1][1])/2, (eig_pairs[1][1][1]+eig_pairs[1][1][1])/2]

        first_cpts = [vec_x[0],vec_y[0]] # more robust

        return first_cpts
        

def isFlipped(thresh_img):
    left = thresh_img[0:thresh_img.shape[0], 0:thresh_img.shape[1]//4]
    right = thresh_img[0:thresh_img.shape[0], (thresh_img.shape[1]-thresh_img.shape[1]//4):thresh_img.shape[1]]
    return len(cvtImageToPoints(left)[0]) < len(cvtImageToPoints(right)[0])

def getAngle(vector):
    from math import atan2, pi
    rad = atan2(vector[1], vector[0])
    deg = rad/(2*pi) * 360
    #print(deg)
    return deg

# angle in degree, positive means clockwise rotation
def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w / 2, h / 2)
    
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH),borderMode=0, borderValue=(255,255,255))

def rotateImg(data_path):
    img = getImg(data_path)
    im = binarizeImg(img)
    first_cpts = getFirstComponent(data_path)

#     rotated = rotate_bound(img1, getAngle(first_cpts))
#     if isFlipped(rotated):
#         first_cpts = [-1*first_cpts[0], -1*first_cpts[1]]
#         rotated = rotate_bound(im, getAngle(first_cpts))

    rotated = rotate_bound(im, getAngle(first_cpts))
    
    return rotated

In [5]:
onlyfiles = [f for f in os.listdir('/Users/i351707/Desktop/useful_python_files/cropped_images_version1_without_cnn/')]
os.chdir('/Users/i351707/Desktop/useful_python_files/cropped_images_version1_without_cnn/')
for file in onlyfiles:
    try:
        rotated = rotateImg(file)

        cv2.imwrite(file.split('.')[0]+'.jpg',rotated)
    except Exception as e:
        print (e)

OpenCV(3.4.1) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:11214: error: (-215) (scn == 3 || scn == 4) && (depth == 0 || depth == 5) in function cvtColor



/Users/i351707/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/i351707/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## try out individually

In [5]:
rotated = rotateImg('1096.png')
cv2.imwrite('results.png',rotated)

True